### This program corresponds to the first and second experiments in the paper and can be used to solve Laplace equations on polygonal domains.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.special as scp
from torch.autograd import Variable as v
import time
import math

import sys
sys.path.append('../Models')
sys.path.append('../Utils')
import importlib

import Network as Nw
import BoundaryCondition as Bdc
import Loss_functions as lf
import DomainSetting as Ds
import GenerateIntegralMatrix as IM
import Training as Tn



In [102]:
# Set problem domain and boundary condition
# Generate integral points on boundary and the integral matrix of the fundamental solution 

# vertex_num  The number of the edges of the polygon
# vertex      Polygon vertices (connected counterclockwise)
# M           Number of samples taken on each edge
# normal      Outer normal vector of each edge
# h           Distance between two points on each side
# sample_num  Total number of samples
# sample_x    Sample point coordinates
# sample_u    Boundary condition

example = 'ex1-1'
if example == 'ex1-1' or example == 'ex1-2':
    
    domaintype = 'polygon'
    problemtype = 'Laplace'
    
    vertex = torch.Tensor([[-1,-1],[1,-1],[1,1],[-1,1]])
    if example == 'ex1-1':
        M = 200
    else:
        M = 500
     
sample_x,normal,h = Ds.polygondomain(M,vertex)

sample_num, vertex_num = len(sample_x), len(vertex) 

sample_u = torch.zeros(sample_num,1)
for i in range(sample_num):
    if (i%M != 0):# and (i%M != 1) and (i%M != M-1):
        sample_u[i] = Bdc.exact_sol(sample_x[i,0],sample_x[i,1],example)

G = IM.G_bd('polygon','Laplace',2,sample_x,h,normal)

In [110]:
# Set neural network, loss fucntion and optimizer
# m neurons num in each layer
m = 100
net1 = Nw.ResNet1(m,2,1) 
loss_func = lf.Laplace_loss()
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0001)

In [122]:
# Training
Epoch, each_epoch = 2000, 1000
net1,loss_all = Tn.laplace_training(net1,sample_x,sample_u,G,loss_func,optimizer,Epoch,each_epoch)

loss, epoch, computation time: 1.130664 0 0.0209
loss, epoch, computation time: 0.000325 1000 18.3147
loss, epoch, computation time: 0.000230 2000 19.5885


In [123]:
# Randomly generate points x_in in the problem domain and the corresponding exact solution
sample = 1000
x_in = (torch.rand(sample,2)*2-1)*0.99
u_in = Bdc.exact_sol(x_in[:,0],x_in[:,1],example).reshape(-1,1)

In [124]:
# Generate integral matrix of points x_in and calculate the relative L2 errir 
G_in = IM.G_in('Laplace',2,x_in,sample_x,h,normal)
u_predict = G2_in@sample_h
print('Relaive L2 error:','%.4f'%((u_green-u_in).norm()/u_in.norm()).detach().numpy())